In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
import torch.optim as optim
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torchvision
from attack import attack, test_model


In [2]:
from datasets import load_cifar10, load_cifar100
from models.resnet import load_cifar10_resnet50, load_cifar100_resnet50
model = load_cifar100_resnet50()


In [3]:
train_loaders, test_dataloaders,train_dataloader_all, test_dataloader_all = load_cifar100()
all_totals = list()
# for i in range(10):
all_totals.append(attack(train_dataloader_all, ["layer3[0].conv2", "layer3[1].conv2", "layer3[5].conv2"], load_cifar100_resnet50, alpha=0.0001))


Files already downloaded and verified
Files already downloaded and verified


100%|██████████| 196/196 [01:14<00:00,  2.62it/s]


0.0025499329730868337


100%|██████████| 196/196 [01:15<00:00,  2.58it/s]


0.003922005892395973


...

0.010425810995101929


100%|██████████| 196/196 [01:20<00:00,  2.43it/s]


0.04959459623336792


100%|██████████| 196/196 [01:20<00:00,  2.44it/s]

0.22819879035949708


In [11]:
all_totals


[{'layer3[0].conv2': array([[[[0.01831029, 0.04096706, 0.06935064],
           [0.01374478, 0.03857579, 0.06768624],
           [0.02512745, 0.04730837, 0.07682443]],
  
          [[0.07613952, 0.08233974, 0.07460476],
           [0.06766784, 0.10873056, 0.13235837],
           [0.0805883 , 0.12918104, 0.16614187]],
  
          [[0.12412132, 0.12345003, 0.12458842],
           [0.08378445, 0.08814042, 0.07941983],
           [0.06012765, 0.0638866 , 0.05912813]],
  
          ...,
  
          [[0.1380302 , 0.18019632, 0.15719634],
           [0.10000233, 0.14583741, 0.14425647],
           [0.06139064, 0.09452016, 0.11249689]],
  
          [[0.07141647, 0.08154185, 0.09171829],
           [0.08127584, 0.08770221, 0.09432267],
           [0.09104168, 0.09530755, 0.09542641]],
  
          [[0.03265534, 0.05615711, 0.08517483],
           [0.02917856, 0.03347686, 0.05923083],
           [0.03479432, 0.02931455, 0.05875172]]],
  
  
         [[[0.13552757, 0.14310564, 0.1503537 ],
    

In [50]:
thre = 0.5
net = load_cifar100_resnet50()
layer_remove = dict()
for layer in ["layer3[0].conv2", "layer3[1].conv2", "layer3[5].conv2"]:
    layer_remove[layer] = None
for i in range(len(all_totals)):
    totals = all_totals[i]
    totals = [totals[layer] for layer in ["layer3[0].conv2", "layer3[1].conv2", "layer3[5].conv2"]]
    layer_weights = [eval("net." + layer + ".weight.cpu().detach().numpy()")
                     for layer in ["layer3[0].conv2", "layer3[1].conv2", "layer3[5].conv2"]]
    combine = [np.abs(total * weight) for total, weight in zip(totals, layer_weights)]
    combine = np.array(combine)
    combine_flatten = combine.flatten()
    threshold = np.sort(combine_flatten)[::-1][int(len(combine_flatten) * thre)]
    for idx,layer in enumerate(["layer3[0].conv2", "layer3[1].conv2", "layer3[5].conv2"]):
        if layer_remove[layer] is None:
            layer_remove[layer] = combine[idx] > threshold
        else:
            t = combine[idx] > threshold
            layer_remove[layer] = layer_remove[layer] | t
    
# for i in range(len(totals)):
#     combine = np.abs(totals[i][layer] * eval("net." + layer + ".weight.cpu().detach().numpy()"))
#     threshold = np.sort(combine.flatten())[::-1][int(len(combine.flatten()) * thre)]
#     if remove is None:
#         remove = combine > threshold
#     else:
#         t = combine > threshold
#         remove = remove | t
# print(remove.sum())
# keep_rate = remove.sum() / remove.size
# print("保留的", keep_rate)


In [51]:
temp = 0
all_num = 0
for layer in layer_remove:
    temp += layer_remove[layer].sum()
    all_num += layer_remove[layer].size
    print(layer, layer_remove[layer].mean())

layer3[0].conv2 0.5980563693576388
layer3[1].conv2 0.3901519775390625
layer3[5].conv2 0.5117916531032987


In [52]:
temp / all_num

0.5

In [53]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    correct, all = test_model(net, test_dataloader_all)
    print("原始准确率", correct / all)


原始准确率 0.7931


In [54]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    for layer in ["layer3[0].conv2", "layer3[1].conv2", "layer3[5].conv2"]:
        exec("net." + layer + ".weight[~layer_remove[layer]] = 0")
    # for layer in ["layer3[0].conv2"]:
    #     exec("net." + layer + ".weight[~remove] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("现在准确率", correct / all)


现在准确率 0.7418


In [55]:
with torch.no_grad():
    net = load_cifar100_resnet50()
    for layer in ["layer3[0].conv2", "layer3[1].conv2", "layer3[5].conv2"]:
        keep_rate = layer_remove[layer].sum() / layer_remove[layer].size
        weight_flatten = eval("net." + layer + ".weight.cpu().detach().numpy()").flatten()
        threshold = np.sort(weight_flatten)[int(len(weight_flatten) * (1 - keep_rate))]
        exec("net." + layer + ".weight[eval('net.' + layer + '.weight.cpu().detach().numpy()') < threshold] = 0")
    correct, all = test_model(net, test_dataloader_all)
    print("去掉最大准确率", correct / all)
    # for layer in ["layer3[0].conv2"]:
    #     threshold = np.sort(eval("net." + layer + ".weight.cpu().detach().numpy()").flatten())[
    #         int(len(eval("net." + layer +
    #             ".weight.cpu().detach().numpy()").flatten()) * (1 - keep_rate))
    #     ]
    #     exec("net." + layer +
    #          ".weight[eval('net.' + layer + '.weight.cpu().detach().numpy()') < threshold] = 0")
    # correct, all = test_model(net, test_dataloader_all)
    # print("现在准确率", correct / all)


去掉最大准确率 0.01
